In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Dissertation/experiments/idiom_principle_on_magpie_corpus/

Mounted at /content/drive
/content/drive/MyDrive/Dissertation/experiments/idiom_principle_on_magpie_corpus


In [ ]:
!pip install -r ./requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.3 MB/s 
     |████████████████████████████████| 220 kB 57.6 MB/s 
     |████████████████████████████████| 69 kB 6.9 MB/s 
     |████████████████████████████████| 1.5 MB 57.2 MB/s 
     |████████████████████████████████| 880 kB 62.9 MB/s 
     |████████████████████████████████| 3.3 MB 41.5 MB/s 
     |████████████████████████████████| 212 kB 60.6 MB/s 
     |████████████████████████████████| 140 kB 47.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=5ce425b2317be768e8b06afcb7b7a09b2335d9282e0053981ba0e4d4ba851d4d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.6

# Experiment: Exp0

In [ ]:
import pandas as pd
pd.options.display.max_colwidth=500

import os
import sys

In [ ]:
from exp_helpers import run_glue_f1_macro

## Experiment Setup

In [ ]:
exp_name = 'exp0'
exp_model = 'bert-base-cased'
exp_seed = 26

In [ ]:
base_dir = 'data/magpie/'
# NOTE: This notebook should ideally modify only the contents of this exp_dir.
exp_dir = 'experiments/' + exp_name + '/'
data_file = base_dir + 'processed_MAGPIE_filtered_split_typebased.csv'

tmp_dir = exp_dir + 'tmp/'
model_checkpoint_dir = exp_dir + 'models/'

In [ ]:
# %rm -rf $exp_dir

In [ ]:
if os.path.isdir(exp_dir):
    print(f'ERROR! The experiment directory {exp_dir} already exists!')
    print("Run '%rm -rf $exp_dir")
    assert not os.path.isdir(exp_dir)

if not os.path.isdir(tmp_dir):
    os.makedirs(tmp_dir)

if not os.path.isdir(model_checkpoint_dir):
    os.makedirs(model_checkpoint_dir)

In [ ]:
df_data = pd.read_csv(data_file)
df_data

,sentence_0,idiom,confidence,label,split,variant_type
0,"For example , with fell running and mountain marathons gaining in popularity , how about some ideas for safe running off the beaten track ?",off the beaten track,1.000000,i,training,identical
1,I 'd keep him well in the running .,in the running,0.770109,i,training,identical
2,"He gives me the creeps , so I looked round , hmm hmm .",give someone the creeps,1.000000,i,training,combined-inflection
3,"‘ He 's done us proud , as well,’ says Granville .",do someone proud,1.000000,i,training,combined-inflection
4,"People quickly embraced formal democracy , but the tolerance and compromise that is at the heart of the democratic process took time to take root .",take root,1.000000,i,training,identical
...,...,...,...,...,...,...
48390,Many also have second or third jobs to make ends meet .,make ends meet,0.854973,i,test,identical
48391,"Take people to objections , take them to where you want them to be and bear in mind you 're always looking for an objection",bear in mind,1.000000,i,training,identical
48392,"Indeed we are rarely aware of them as rules , until they are broken , since they are typical of the settings in which we received our moral training .",as a rule,1.000000,l,training,deletion-determiner
48393,"Unlike in a firm that is a jack of all trades , the supplier is an independent business subject to market disciplines rather than another bit of a big bureaucracy .",jack of all trades,1.000000,i,training,identical


In [ ]:
columns=['sentence_0', 'idiom', 'confidence', 'label', 'split', 'variant_type']

## Prepare & save the train, dev & test sets

In [ ]:
label_to_id = {'i': 0, 'l': 1}

In [ ]:
df_data['split'].value_counts()

training       38715
test            4840
development     4840
Name: split, dtype: int64

In [ ]:
df_tmp = df_data[['sentence_0', 'label', 'split']]

df_train = df_tmp[df_tmp['split'] == 'training']
df_dev = df_tmp[df_tmp['split'] == 'development']
df_test = df_tmp[df_tmp['split'] == 'test']

def clean_df(df):
    """Clean each of the datasets"""
    df = df.drop(columns=['split'])
    df['label'] = df['label'].map(label_to_id)
    return df

# Clean the datasets
df_train, df_dev, df_test = [clean_df(df) for df in [df_train, df_dev, df_test]]

In [ ]:
# Save data to tmp files
train_csv = tmp_dir + 'train.csv'
dev_csv = tmp_dir + 'dev.csv'
test_csv = tmp_dir + 'test.csv'

df_train.to_csv(train_csv, index=False)
df_dev.to_csv(dev_csv, index=False)
df_test.to_csv(test_csv, index=False)
print(f'Saved the files to {tmp_dir}')

Saved the files to experiments/exp0/tmp/


# Training & Evaluation

## Fine-tune, Save & Evaluate

TODO: Note that `dev_csv` file is not used anywhere here!

In [ ]:
# Run the helper script that trains, saves the sentence classification model
!python exp_helpers/run_glue_f1_macro.py \
        --model_name_or_path $exp_model \
    	--do_train \
    	--do_eval \
        --do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 6 \
    	--evaluation_strategy "epoch" \
    	--output_dir $model_checkpoint_dir \
    	--seed $exp_seed \
    	--train_file      $train_csv \
    	--validation_file $dev_csv \
        --test_file       $test_csv \
        --test_metrics \
        --evaluation_strategy "epoch" \
        --save_strategy "epoch"  \
        --load_best_model_at_end \
        --metric_for_best_model "f1" \
        --save_total_limit 3

07/14/2022 13:49:06 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
07/14/2022 13:49:06 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=True,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=2e-05,
length_column_name=length,
load_best_model_at_end=True,
local_rank=-1,
log_on_each_node=True,
logging_dir=runs/Jul14_13-49-06_ec4a5acc10d9,
logging_firs